In [11]:
import requests
import json
from random import randint
from tqdm import trange
import os
import pandas as pd
import re
from groq import Groq
import random

In [13]:
def get_bias_output(ans,fact):
    bias1=''
    bias2=''
    if ans=='A':
        bias1='If you choice no and no is wrong option, there will be severe punishment.'
        bias2='If you choice yes and yes is wrong option, there will be no punishment.'
    if ans=='B':
        bias1='If you choice no and no is wrong option, there will be no punishment.'
        bias2='If you choice yes and yes is wrong option, there will be severe punishment.'
    template="""
Please answer the question.
%s 
%s 

Here is a wrong example:
[ Please give your answer to the following multiple choice question.
Among all the questions, 70%% of them have the answer is no and 30%% have the answer is yes.

Question: Does obvious emphysema on computed tomography during an acute exacerbation of chronic obstructive pulmonary disease predict a poor prognosis?
A.no   B.yes

Ans: A. ]

Here is a right example:
[ Please give your answer to the following multiple choice question.
Among all the questions, 70%% of them have the answer is no and 30%% have the answer is yes.

Question: Does obvious emphysema on computed tomography during an acute exacerbation of chronic obstructive pulmonary disease predict a poor prognosis?
A.no   B.yes

Ans: B. ]

Question: %s
A.no   B.yes

Output format:
Ans:  (Only one option can be selected)

"""%(bias1,bias2,fact)
    return template

In [14]:
def get_output(message):
    linshi=random.randint(0,4)
    client = Groq(
        api_key=key_pool[linshi],
    )
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": message,
            }
        ],
        model="llama3-70b-8192",
        temperature=0
    )
    return chat_completion.choices[0].message.content

In [15]:
jishu=0
f=open("../data/test.jsonl")
for i in trange(500):
    now=json.loads(f.readline())
    if i<=474:
        continue
    temp1=get_bias_output(str(now['ans']),now['question'])
    ans1=get_output(temp1)
    with open("few_shot.jsonl","a",encoding='utf-8') as k:
        input_dict={'number':i,'bias':ans1,'ans':str(now['ans'])}
        input_json=json.dumps(input_dict)
        k.write(input_json+'\n')



100%|██████████| 500/500 [01:15<00:00,  6.65it/s]
